# 1 - Data preparations

# Backround

The article uses the highly used and preprocessed bioactivity dataset from the ChEMBL database, version 20. More specifically these seven different bioactivity classes:

(a) Carbonic Anhydrase II (ChEMBL205), a protein lyase,  
(b) Cyclin-dependent kinase 2 (CHEMBL301), a protein kinase,  
(c) ether-a-go-go-related gene potassium channel 1 (HERG) (CHEMBL240), a voltage-gated ion channel,  
(d) Dopamine D4 receptor (CHEMBL219), a monoamine GPCR,  
(e) Coagulation factor X (CHEMBL244), a serine protease,  
(f) Cannabinoid CB1 receptor (CHEMBL218), a lipid-like GPCR and  
(g) Cytochrome P450 19A1 (CHEMBL1978), a cytochrome P450.  
The activity classes were selected based on data availability and as representatives of therapeutically important target classes or as anti-targets.

In this project we will mainly focus on (a) the ChEMBL205 and (f) ChEMBL218 as these got quite different results from each other. Also 

In [1]:
#hide
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
#hide
from rdkit import Chem
from rdkit.Chem import AllChem

ModuleNotFoundError: No module named 'rdkit'

In [10]:
#hide
path = Path('../dataset/13321_2017_226_MOESM1_ESM/')
#df = pd.read_csv('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl.csv', index_col=0)

In [11]:
#hide
#df.head()
list(path.iterdir())

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/13321_2017_226_MOESM1_ESM'

First we need to convert the smiles from the dataset to ECFP fingerprint which is hashed into 1024 length bits. For this we made a function for creating a fingerprint using the methods from the RDKit library as follows: 

In [5]:
def fp(smile, diam = 2, bits = 1024):

    mol = Chem.MolFromSmiles(smile)
    Chem.SanitizeMol(mol)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, diam, nBits = bits) 
    return fp

We then define a method for converting the csv with smiles to also have the fingerprint stored as colums from ECFP_1 to ECFP_1024 for each row. 

In [6]:
#ECFP4
#Generated Circular fingerprints hashed into n bits length vectors.

def ECFP(ifile, ofile, diam, bits):
    
    print(f"Making fingerprints for file: {ifile}")
    df = pd.read_csv(ifile)
    
    df.insert(2, "ECFP4_", df.SMILES.apply(fp))
    
    df[[f"ECFP4_{i+1}" for i in range(len(df.ECFP4_[0]))]] = df.ECFP4_.to_list()
    
    df.drop("ECFP4_", axis = 1, inplace = True)
    
    
    df.to_csv(path/ofile, index = None)
    return df

Here we can specify the dataset we want to use and to run the ECFP function on. 

In [7]:
dataset='CHEMBL205_cl'

In [8]:
#hide_output
#ECFP(path/f'{dataset}.csv', f'./{dataset}_ecfp_1024.csv', 2, 1024)

Next we create a dataframe from the newly generated csv file. 

In [9]:
df = pd.read_csv(path/f'{dataset}_ecfp_1024.csv')

NameError: name 'path' is not defined

In [3]:
df.head()

NameError: name 'df' is not defined

In [1]:
df.info()

NameError: name 'df' is not defined

In [9]:
X, y = df.drop(["CID", "SMILES", "Activity"], axis=1), df["Activity"]

In [10]:
#hide
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17941 entries, 0 to 17940
Columns: 1024 entries, ECFP4_1 to ECFP4_1024
dtypes: int64(1024)
memory usage: 140.2 MB


In [11]:
#hide
y.head(), y.size, type(y)

(0    1
 1    1
 2    1
 3    1
 4    1
 Name: Activity, dtype: int64,
 17941,
 pandas.core.series.Series)

# Train test split

In [12]:
#hide
from sklearn.model_selection import train_test_split, KFold

In [13]:
# regular train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

# 5-Fold Cross Validation

Follows the article where the data is split using 5-fold cross validation. 

In [14]:
#5-fold
kf = KFold(n_splits=5, shuffle=True, random_state=999)

In [15]:
X_train_list, X_valid_list, y_train_list, y_valid_list = [], [], [], []

for train_index, valid_index in kf.split(X_train):
    X_train_list.append(X_train.iloc[train_index])
    X_valid_list.append(X_train.iloc[valid_index])
    y_train_list.append(y_train.iloc[train_index])
    y_valid_list.append(y_train.iloc[valid_index])

In [16]:
#hide
y_train_list[0].head()

14960    0
16280    0
16068    0
13692    0
14043    0
Name: Activity, dtype: int64

In [18]:
#hide
X_train_list[1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8611 entries, 14960 to 10114
Columns: 1024 entries, ECFP4_1 to ECFP4_1024
dtypes: int64(1024)
memory usage: 67.3 MB


# Random Forest

In [26]:
#hide
from sklearn.ensemble import RandomForestClassifier

In [27]:
#hide
from sklearn.metrics import auc,roc_auc_score,recall_score,precision_score,f1_score
from  sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

In [41]:
# train method for Random Forest
def train_rf(X_train, X_test, y_train, y_test, n_estimators=5, criterion='gini', max_features='log2'):

    
    rf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, min_samples_split=2, max_features=max_features, 
                               max_leaf_nodes=None,bootstrap=False,oob_score=False, n_jobs=-1, random_state=69)
    
    rf.fit(X_train,y_train)
    y_pred= rf.predict(X_test)
    y_pred_prob=rf.predict_proba(X_test)
    
    temp=[]
    for j in range(len(y_pred_prob)):
        temp.append(y_pred_prob[j][1])
    auc=roc_auc_score(np.array(y_test),np.array(temp))
    acc2=accuracy_score(y_test,y_pred)
    mcc=matthews_corrcoef(y_test,y_pred)
    Recall=recall_score(y_test, y_pred,pos_label=1)
    Precision=precision_score(y_test, y_pred,pos_label=1)
    F1_score=f1_score(y_test, y_pred,pos_label=1)

    return auc,acc2,mcc,Recall,Precision,F1_score

In [42]:


# Train and print scores for each split
for i in range(0,5):
    
    print(f'Cross validation iteration: {i + 1}')
    
    X_train = X_train_list[i]
    X_valid = X_valid_list[i]
    y_train = y_train_list[i]
    y_valid = y_valid_list[i]
    
    auc,acc2,mcc,Recall,Precision,F1_score = train_rf(X_train, X_valid, y_train, y_valid, 
                                                      n_estimators=100, criterion='entropy', max_features='sqrt')
    
    print(f'AUC score:{auc}')
    print(f'Accuracy: {acc2}')
    print(f'Matthews: {mcc}')
    print(f'Recall: {Recall}')
    print(f'Precision: {Precision}')
    print(f'F1 score: {F1_score}')
    print()

Cross validation iteration: 1
AUC score:0.9887427620260744
Accuracy: 0.9730608453320948
Matthews: 0.8415714713449818
Recall: 0.8564356435643564
Precision: 0.8564356435643564
F1 score: 0.8564356435643564

Cross validation iteration: 2
AUC score:0.9876712143359863
Accuracy: 0.9781699953553181
Matthews: 0.8608807076715227
Recall: 0.8563829787234043
Precision: 0.8895027624309392
F1 score: 0.8726287262872627

Cross validation iteration: 3
AUC score:0.9909322903664254
Accuracy: 0.9712029725963771
Matthews: 0.8122047591950275
Recall: 0.8142076502732241
Precision: 0.8418079096045198
F1 score: 0.8277777777777778

Cross validation iteration: 4
AUC score:0.9754214676396435
Accuracy: 0.9693450998606595
Matthews: 0.8300350847464447
Recall: 0.7964601769911505
Precision: 0.9
F1 score: 0.8450704225352113

Cross validation iteration: 5
AUC score:0.9912994366040052
Accuracy: 0.9753717472118959
Matthews: 0.8390326671715927
Recall: 0.8406593406593407
Precision: 0.864406779661017
F1 score: 0.85236768802228